In [1]:
from hoshi_workflow import make_initial_models as mim
import importlib
importlib.reload(mim)
mim.enable_logging()
import numpy as np
from datetime import datetime

In [2]:
import logging
mim.enable_logging(level=logging.INFO)  # set logging level to INFO, the default is INFO

# or one can disable logging
# mim.disable_logging()

In [3]:
# initialize HoshiModelDir, where to make initial models
# here we use a fake model directory for testing
path = "./example_model/fake_model"
hm = mim.HoshiModelDir(path)
print(hm.henyey_params)


{'input_structure_dir': 'strdata', 'output_summary_dir': 'summary', 'input_structure_file': 'M11F2d00_Z05F1d00', 'output_structure_file': 'M11F2d00_Z00F0d00', 'NAME_IN': 'e', 'NAME_OUT': 'e', 'NSTG_max': '1234', 'MAXQTY': '1.d10\t2', 'FLAG_reduce_D_and_Li': '1', 'relative_metallicity': '0.0', 'FLAG_change_mass': '1', 'new_mass': '20.0'}


In [4]:

# to execute 'evol' in the model directory, but here fake model is not able to run evol 
# one should test with real model directory
# if evol successfully run, an exit code 0 will be returned
#hm.run()

# fake run to create a mock output initial structure file, just for testing
hm.fake_run()

2025-11-05 14:08:10,669 INFO hoshi_workflow.make_initial_models.make_initial_models: Running fake simulation with current parameters:
 {'input_structure_dir': 'strdata', 'output_summary_dir': 'summary', 'input_structure_file': 'M11F2d00_Z05F1d00', 'output_structure_file': 'M11F2d00_Z00F0d00', 'NAME_IN': 'e', 'NAME_OUT': 'e', 'NSTG_max': '1234', 'MAXQTY': '1.d10\t2', 'FLAG_reduce_D_and_Li': '1', 'relative_metallicity': '0.0', 'FLAG_change_mass': '1', 'new_mass': '20.0'}


0

In [5]:
new_henyey_params = {
    # 'input_structure_dir': 'strdata',
    # 'output_summary_dir': 'summary',
    'input_structure_file': 'M11F1d50_Z05F1d00',
    'output_structure_file': 'test',
    'NAME_IN': 'e',
    'NAME_OUT': 'e',
    'NSTG_max': '1234',
    'MAXQTY': '1.d10\t2',
    'FLAG_reduce_D_and_Li': '1',
    'relative_metallicity': '5.d-6',
    'FLAG_change_mass': '1',
    'new_mass': '3.56d1',
}

# one can use modify_input_henyey to change henyey_params
hm.modify_input_henyey(
    new_params=new_henyey_params, 
    backup=True, # backup the original Henyey.in file
    dry_run=False # if True, do not actually write the changes
)

print(hm.henyey_params)


2025-11-05 14:08:10,675 INFO hoshi_workflow.make_initial_models.make_initial_models: Backup created: example_model/fake_model/param/files.henyey.bak
2025-11-05 14:08:10,676 INFO hoshi_workflow.make_initial_models.make_initial_models: Updated Henyey file written: example_model/fake_model/param/files.henyey


{'input_structure_dir': 'strdata', 'output_summary_dir': 'summary', 'input_structure_file': 'M11F1d50_Z05F1d00', 'output_structure_file': 'test', 'NAME_IN': 'e', 'NAME_OUT': 'e', 'NSTG_max': '1234', 'MAXQTY': '1.d10\t2', 'FLAG_reduce_D_and_Li': '1', 'relative_metallicity': '5.d-6', 'FLAG_change_mass': '1', 'new_mass': '3.56d1'}


In [6]:
# now use make_initial_model to create initial models
# original structure file, target mass and metallicity need to be specified
# output files will be named automatically, based on target mass and metallicity, following the naming convention
hm.make_initial_model(
    initial_structure_file='M11F1d50_Z10F1d00', # must follow the naming convention
    target_relative_metallicity=0.0,
    target_mass=20.,
    dry_run=True, # use fake_run instead of run for testing
)

2025-11-05 14:08:12,980 INFO hoshi_workflow.make_initial_models.make_initial_models: Updating input_structure_file to M11F1d50_Z10F1d00
2025-11-05 14:08:12,983 INFO hoshi_workflow.make_initial_models.make_initial_models: Backup created: example_model/fake_model/param/files.henyey.bak
2025-11-05 14:08:12,984 INFO hoshi_workflow.make_initial_models.make_initial_models: Updated Henyey file written: example_model/fake_model/param/files.henyey
2025-11-05 14:08:12,985 INFO hoshi_workflow.make_initial_models.make_initial_models: Making initial model, 6 loops left...
2025-11-05 14:08:12,987 INFO hoshi_workflow.make_initial_models.make_initial_models: Backup created: example_model/fake_model/param/files.henyey.bak
2025-11-05 14:08:12,989 INFO hoshi_workflow.make_initial_models.make_initial_models: Updated Henyey file written: example_model/fake_model/param/files.henyey
2025-11-05 14:08:12,990 INFO hoshi_workflow.make_initial_models.make_initial_models: Running fake simulation with current param

The mass change is applied in the first simulation.  
The relative initial metallicity, $Z_{rel}$, is varied in logarithmic steps, with each step being at most one order of magnitude.  
For cases where the target $Z_{rel} < 10^{-5}$, or for the zero-metallicity ($Z=0$) case, the sequence first steps down to $Z_{rel} = -5$ and then jumps directly to the final target value.  

For the test case above, you will see the following output files:

```Bash
/HOSHI_WorkFlow/examples/example_model/fake_model/strdata$ls
M11F2d00_Z00F0d00.fake  M11F2d00_Z05F1d00.fake  M11F2d00_Z06F1d00.fake  
M11F2d00_Z07F1d00.fake  M11F2d00_Z08F1d00.fake  M11F2d00_Z09F1d00.fake
```

Additionally, a log file for each simulation will be generated in the HOSHI working directory, following the format <run_YY_MMDD_HHMM.log>.Here, `YY` represents the two-digit year, while `MMDD` and `HHMM` represent the four-digit date and time, respectively.